In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
import random

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [2]:
(train_dataset,test_dataset),ds_info = tfds.load('imdb_reviews',
                                                 with_info = True,
                                         split = ['train','test'],
                                                  as_supervised = True,
                                         shuffle_files = True,
                                         batch_size = 128

                                                 )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHBRPOI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHBRPOI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHBRPOI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
def text_vectorization(dataset,vectorizer):
  """
  Uses TextVectorization Layer outside of the model.
  The purpose is to make the NN model suitable to .h5 format.

  Parameters
  ----------
  dataset: Tensorflow dataset containing data and labels.
  vectorizer: A TextVectorization layer adapted to train_dataset
  """
  #conduct all the process in CPU to CPU-GPU conflict
  with tf.device("/cpu:0"):
    outputs = []
    label_list = []

    #tokenize and pad the data and store it
    for x, y in dataset:
        output = vectorizer(x)
        outputs.append(output)
        label_list.append(y)

    #concatenate the labels and data
    X_vectorized = tf.concat(outputs, axis=0)
    Y_labels = tf.concat(label_list, axis=0)

    vectorized = tf.data.Dataset.from_tensor_slices((X_vectorized, Y_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
    return vectorized

def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 5,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")

  return es,mc,tb

In [4]:
#get the vocab length
vectorizer = layers.TextVectorization(max_tokens = None)
vectorizer.adapt(train_dataset.map(lambda x,y: x))
total_words = vectorizer.vocabulary_size()

In [5]:
#save the length of the sequences to find the output sequence length
text_lengths = []
train_dataset_unbatched = train_dataset.unbatch()
for text,label in train_dataset_unbatched: #unbatch the dataset
  text_lengths.append(len(text.numpy().split())) #append the sentence length

In [6]:
#create model and history dictionaries
models = {}
histories = {}

In [7]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model1"
HISTORY_NAME = 'history1'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)

vectorizer.adapt(train_dataset.map(lambda x,y: x))
#preprocess the data
train_dataset_vectorized1 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized1 = text_vectorization(test_dataset,vectorizer)


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized1,validation_data = test_dataset_vectorized1,epochs = 5,
                    )



Epoch 1/5
782/782 [==============================] - 17s 21ms/step - loss: 0.6052 - accuracy: 0.7125 - val_loss: 0.4866 - val_accuracy: 0.8170
Epoch 2/5
782/782 [==============================] - 16s 21ms/step - loss: 0.3873 - accuracy: 0.8642 - val_loss: 0.3591 - val_accuracy: 0.8659
Epoch 3/5
782/782 [==============================] - 17s 22ms/step - loss: 0.2920 - accuracy: 0.8956 - val_loss: 0.3152 - val_accuracy: 0.8805
Epoch 4/5
782/782 [==============================] - 16s 21ms/step - loss: 0.2424 - accuracy: 0.9127 - val_loss: 0.2953 - val_accuracy: 0.8858
Epoch 5/5
782/782 [==============================] - 17s 21ms/step - loss: 0.2081 - accuracy: 0.9272 - val_loss: 0.2862 - val_accuracy: 0.8877


Add another Dense Layer: Not worked. Increased overfitting.

In [8]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model2"
HISTORY_NAME = 'history2'


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized1,validation_data = test_dataset_vectorized1,epochs = 5,
                   )


Epoch 1/5
782/782 [==============================] - 18s 23ms/step - loss: 0.4431 - accuracy: 0.7888 - val_loss: 0.2993 - val_accuracy: 0.8814
Epoch 2/5
782/782 [==============================] - 17s 22ms/step - loss: 0.2207 - accuracy: 0.9140 - val_loss: 0.2841 - val_accuracy: 0.8850
Epoch 3/5
782/782 [==============================] - 16s 21ms/step - loss: 0.1577 - accuracy: 0.9423 - val_loss: 0.2974 - val_accuracy: 0.8852
Epoch 4/5
782/782 [==============================] - 16s 21ms/step - loss: 0.1179 - accuracy: 0.9588 - val_loss: 0.3413 - val_accuracy: 0.8780
Epoch 5/5
782/782 [==============================] - 17s 21ms/step - loss: 0.0936 - accuracy: 0.9691 - val_loss: 0.3997 - val_accuracy: 0.8709


Decrese the tokens: Worked. Overfitting decreased.

In [9]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model3'
HISTORY_NAME = 'history3'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
#x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 5)

Epoch 1/5
782/782 [==============================] - 6s 7ms/step - loss: 0.6397 - accuracy: 0.6916 - val_loss: 0.5590 - val_accuracy: 0.7758
Epoch 2/5
782/782 [==============================] - 6s 7ms/step - loss: 0.4680 - accuracy: 0.8355 - val_loss: 0.4216 - val_accuracy: 0.8464
Epoch 3/5
782/782 [==============================] - 6s 8ms/step - loss: 0.3612 - accuracy: 0.8732 - val_loss: 0.3582 - val_accuracy: 0.8660
Epoch 4/5
782/782 [==============================] - 6s 8ms/step - loss: 0.3062 - accuracy: 0.8886 - val_loss: 0.3260 - val_accuracy: 0.8757
Epoch 5/5
782/782 [==============================] - 7s 8ms/step - loss: 0.2716 - accuracy: 0.9009 - val_loss: 0.3075 - val_accuracy: 0.8812


Try Flattening: Not worked Increased overfitting a lot.

In [10]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model4'
HISTORY_NAME = 'history4'

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.Flatten()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 5)



Epoch 1/5
782/782 [==============================] - 6s 7ms/step - loss: 0.4395 - accuracy: 0.7878 - val_loss: 0.3097 - val_accuracy: 0.8708
Epoch 2/5
782/782 [==============================] - 5s 7ms/step - loss: 0.1885 - accuracy: 0.9320 - val_loss: 0.3147 - val_accuracy: 0.8710
Epoch 3/5
782/782 [==============================] - 5s 7ms/step - loss: 0.0838 - accuracy: 0.9791 - val_loss: 0.3504 - val_accuracy: 0.8644
Epoch 4/5
782/782 [==============================] - 5s 7ms/step - loss: 0.0316 - accuracy: 0.9964 - val_loss: 0.3905 - val_accuracy: 0.8602
Epoch 5/5
782/782 [==============================] - 6s 7ms/step - loss: 0.0124 - accuracy: 0.9994 - val_loss: 0.4249 - val_accuracy: 0.8593


Use Convolution Layer:

In [11]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model5'
HISTORY_NAME = 'history5'
ACTIVATION = 'swish'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.Conv1D(filters = 32,kernel_size = 3,activation = ACTIVATION)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 100,
                                                 callbacks = [es,mc,tb])

Epoch 1/100
782/782 [==============================] - 21s 26ms/step - loss: 0.4641 - accuracy: 0.7778 - val_loss: 0.3148 - val_accuracy: 0.8781
Epoch 2/100
782/782 [==============================] - 21s 27ms/step - loss: 0.2524 - accuracy: 0.9035 - val_loss: 0.2869 - val_accuracy: 0.8867
Epoch 3/100
782/782 [==============================] - 20s 25ms/step - loss: 0.1981 - accuracy: 0.9266 - val_loss: 0.2938 - val_accuracy: 0.8862
Epoch 4/100
782/782 [==============================] - 19s 25ms/step - loss: 0.1629 - accuracy: 0.9412 - val_loss: 0.3161 - val_accuracy: 0.8806
Epoch 5/100
782/782 [==============================] - 19s 25ms/step - loss: 0.1369 - accuracy: 0.9519 - val_loss: 0.3456 - val_accuracy: 0.8765
Epoch 6/100
782/782 [==============================] - 22s 28ms/step - loss: 0.1160 - accuracy: 0.9614 - val_loss: 0.3821 - val_accuracy: 0.8712
Epoch 7/100
782/782 [==============================] - 20s 26ms/step - loss: 0.0987 - accuracy: 0.9683 - val_loss: 0.4239 - val_ac

Train model 3 for longer.

In [12]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model6'
HISTORY_NAME = 'history6'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 100,
                                                 callbacks = [es,mc,tb])

Epoch 1/100
782/782 [==============================] - 7s 8ms/step - loss: 0.6404 - accuracy: 0.6913 - val_loss: 0.5611 - val_accuracy: 0.7748
Epoch 2/100
782/782 [==============================] - 6s 7ms/step - loss: 0.4704 - accuracy: 0.8345 - val_loss: 0.4235 - val_accuracy: 0.8460
Epoch 3/100
782/782 [==============================] - 6s 7ms/step - loss: 0.3630 - accuracy: 0.8724 - val_loss: 0.3594 - val_accuracy: 0.8656
Epoch 4/100
782/782 [==============================] - 6s 8ms/step - loss: 0.3075 - accuracy: 0.8886 - val_loss: 0.3268 - val_accuracy: 0.8752
Epoch 5/100
782/782 [==============================] - 6s 8ms/step - loss: 0.2727 - accuracy: 0.9004 - val_loss: 0.3081 - val_accuracy: 0.8814
Epoch 6/100
782/782 [==============================] - 6s 7ms/step - loss: 0.2474 - accuracy: 0.9100 - val_loss: 0.2966 - val_accuracy: 0.8836
Epoch 7/100
782/782 [==============================] - 6s 7ms/step - loss: 0.2273 - accuracy: 0.9178 - val_loss: 0.2899 - val_accuracy: 0.8854

Add dropout and Dense layer. Change Activation to swish

In [13]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model7'
HISTORY_NAME = 'history7'
ACTIVATION = 'swish'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized2 = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized2 = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation = ACTIVATION)(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(train_dataset_vectorized2,validation_data = test_dataset_vectorized2,epochs = 100,
                                                 callbacks = [es,mc,tb])

Epoch 1/100
782/782 [==============================] - 8s 9ms/step - loss: 0.4889 - accuracy: 0.7541 - val_loss: 0.3188 - val_accuracy: 0.8744
Epoch 2/100
782/782 [==============================] - 6s 7ms/step - loss: 0.2575 - accuracy: 0.9002 - val_loss: 0.2851 - val_accuracy: 0.8871
Epoch 3/100
782/782 [==============================] - 6s 7ms/step - loss: 0.2035 - accuracy: 0.9222 - val_loss: 0.2901 - val_accuracy: 0.8872
Epoch 4/100
782/782 [==============================] - 6s 8ms/step - loss: 0.1704 - accuracy: 0.9360 - val_loss: 0.3140 - val_accuracy: 0.8818
Epoch 5/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1468 - accuracy: 0.9460 - val_loss: 0.3514 - val_accuracy: 0.8761
Epoch 6/100
782/782 [==============================] - 6s 8ms/step - loss: 0.1264 - accuracy: 0.9559 - val_loss: 0.3905 - val_accuracy: 0.8719
Epoch 7/100
782/782 [==============================] - 6s 7ms/step - loss: 0.1098 - accuracy: 0.9630 - val_loss: 0.4396 - val_accuracy: 0.8686

NOTE: LSTM does not work in GPU when it is used with relu;

In [14]:
#!tensorboard dev upload --logdir /content/TensorboardLogs

In [15]:
#clone the model
final_model = tf.keras.models.clone_model(models['model5'])

#load the best weights
final_model.load_weights("/content/ModelCheckpoints/model5.ckpt")

#compile the model
final_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#evaluate the test dataset
final_model.evaluate(test_dataset_vectorized2)

782/782 [==============================] - 5s 6ms/step - loss: 0.2869 - accuracy: 0.8867


[0.28687766194343567, 0.886680006980896]

In [16]:
#!tensorboard dev list

In [17]:
#!tensorboard dev delete --experiment_id rvOlSmxeQMiBvT4XhMmLdw

In [18]:
#final_model.save('final_model.h5')